# Running Computational Alchemy for Catalysis: Transition State Barrier Height Predictions
This notebook outlines Python functions that use information from VASP calculations to predict descriptors for hypothetical catalyst. These functions utilize tools from the Atomic Simulation Environment (ASE) to manipulate catalyst surface models and perform calculations with computational alchemy. 

In this example, we apply computational alchemy on reference set of nudged elastic band (NEB) calculcations of CH4 dehydrogenation on Pt(111) to predict transition state barrier heights of the same process on hypothetical alloys of Pt(111). 

In [1]:
import comp_alchemy

In [2]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, CustomJS, Range1d
from bokeh.models.annotations import Slope
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook

This example uses an NEB calculation with 10 images in total. Below we plot the total energy for each image. You also can hover your mouse over any data point to visualize the image

In [3]:
images = range(10)
image_energies = pd.read_csv('reference_images_energies.csv')
image_energies = image_energies['energy']

In [4]:
output_notebook()

en_source = ColumnDataSource(
data=dict(
x=[int(im) for im in images],
y=image_energies,
ads_imgs=['neb_images/'+str(im)+'/'+'neb_image_'+str(im)+'.png' for im in images]))

en_tooltips="""
            <p>
                <span style="font-size: 25px;">NEB Image </span>
                <span style="font-size: 25px;">$index</span>
            </p>
            <p>
                <img
                src="@ads_imgs" height="150" alt="@ads_imgs" width="150"
                style="float: center; margin: 0px 15px 15px 0px;"
                border="2"
                ></img>
"""

ef = figure(plot_width=900, plot_height=600,y_range=(min(image_energies)-0.5,max(image_energies)+0.5),tooltips=en_tooltips)

ef.xaxis.axis_label = "NEB Image Number"
ef.xaxis.axis_label_text_font_style = "bold"
ef.xaxis.axis_label_text_font_size = "12pt"
ef.yaxis.axis_label = "Energy (eV)"
ef.yaxis.axis_label_text_font_style = "bold"
ef.yaxis.axis_label_text_font_size = "12pt"
ef.title.text_font_size = '12pt'
ef.title.align = 'center'

ef.circle('x', 'y', size=45, source=en_source)
ef.line('x','y', line_dash="dashed", source=en_source)

show(ef, notebook_handle=True)

Loading BokehJS ...

From the plot above, it's apparent that image 5 is the transition state for CH4 dehydrogenation.

Next we make ASE Atoms objects with VASP POSCARs and CONTCARs for the bare catalyst surface (slab) and the catalyst surface with the adsorbate (ads) for each image. These models and the results of the VASP calculations for each will be used as our reference for computational alchemy calculations.

In [5]:
from ase.io import read 
from ase.visualize import view
slab = read('slab/POSCAR',format='vasp')
slab_contcar = read('slab/CONTCAR',format='vasp')
ads = [read(f'neb_images/{str(im)}/POSCAR',format='vasp') for im in images]
ads_contcar = [read(f'neb_images/{str(im)}/CONTCAR',format='vasp') for im in images]

In [6]:
from comp_alchemy.ads_slab_pairs import pairs
p = [pairs(slab,slab) for a in ads]

### Working with electrostatic potentials in VASP OUTCAR
Next we can grab the atom-centered electrostatic potentials (ESPs) printed at the end of the OUTCAR with `grab_esp()`. 
Here we grab ESPs for slab and ads and calculate the differences with `espdiff()`. The ESP differnces define the alchemical derivatives that facilitate predictions with alchemy.

In [7]:
from comp_alchemy.elec_stat_pot import (grab_esp,espdiff,remove_duplicate_espdiffs,heatmap)
slab_elec = grab_esp(poscar='slab/POSCAR',outcar='slab/trunc-OUTCAR')
ads_elec = [grab_esp(poscar=f'neb_images/{str(im)}/POSCAR',
                     outcar=f'neb_images/{str(im)}/trunc-OUTCAR') for im in images]
diffs = [espdiff(elec1=slab_elec,elec2=ads_elec[im],pair=p[im]) for im in images]

In [8]:
diffs0 = [round(d,4) for d in diffs[0]]
print(f'''
Atom-centered electrostatic potentials for the initial state of CH4 dehydrogenation (image 0):

Slab:
{slab_elec}

Ads:
{ads_elec[0]}

Differences in electrostatic potentials between ads and slab
{diffs0}''')


Atom-centered electrostatic potentials for the initial state of CH4 dehydrogenation (image 0):

Slab:
[-73.7064, -73.7064, -73.7064, -73.7064, -74.1493, -74.1493, -74.1493, -74.1493, -74.0957, -74.0957, -74.0957, -74.0957, -73.6192, -73.6192, -73.6192, -73.6192]

Ads:
[-74.1863, -74.186, -74.1864, -74.1866, -74.6565, -74.6556, -74.6567, -74.6575, -74.6527, -74.6519, -74.6519, -74.6528, -74.1955, -74.1747, -74.1757, -74.1765, -53.2355, -36.6325, -36.6332, -36.6324, -36.6331]

Differences in electrostatic potentials between ads and slab
[-0.4799, -0.4796, -0.48, -0.4802, -0.5072, -0.5063, -0.5074, -0.5082, -0.557, -0.5562, -0.5562, -0.5571, -0.5763, -0.5555, -0.5565, -0.5573]


### Visualizing alchemical derivatives
With the `heatmap()` function, visualize the alchemical derivatives with ASE GUI by running the cell below, pressing "c" in ASE GUI, and under "Choose how the atoms are colored:" selecting "By initial charge" 

In [9]:
slab_vis = [heatmap(poscar='slab/POSCAR',dexlist=[dex[0] for dex in p[im]],espdiffs=diffs[im]) for im in range(0,len(images))]
#view(slab_vis)

### Creating hypothetical materials by making transmutations
Below we use the `index_transmuted()` function to grab indexes of atoms that we want to transmute in our reference slabs. We specify the number of transmutable atoms by setting `transmute_num`. 

This function also grabs indexes of atoms at the bottom of the slab that we can counter-transmute to maintain isoelectronicity. We set the number of counter-transmutable atoms with `counter_num`.

In [10]:
from ase import Atom
from comp_alchemy.alloy_index import index_transmuted, transmuter, transmuted_directory_names
metal = Atom('Pt')
[transmute, counter] = index_transmuted(slab=slab,
                                        transmute_atom_sym=metal.symbol,
                                        counter_atom_sym=metal.symbol,
                                        transmute_num=8,
                                        counter_num=2,
                                        symmetric = False)

transmute = [transmute for im in images]
counter = [counter for im in images]

In [11]:
print(f'''Indices of atoms to be transmuted in the top two surface layers.
{transmute[0]}

Indices of atoms to be counter transmuted with unique electrostatic potential differences.
{counter[0]}

There are {len(transmute)} sites near the surface of the catalyst where we will make transmutations.

For each transmutation, we can make counter transmutations at 1 of {len(counter)} sites.

Considering all combinations of transmute and counter-transmute sites, we will assess a total of {len(transmute)*len(counter)} hypothetical configurations of alloys.
''')

Indices of atoms to be transmuted in the top two surface layers.
[12, 13, 14, 15, 8, 9, 10, 11]

Indices of atoms to be counter transmuted with unique electrostatic potential differences.
[0, 1]

There are 10 sites near the surface of the catalyst where we will make transmutations.

For each transmutation, we can make counter transmutations at 1 of 10 sites.

Considering all combinations of transmute and counter-transmute sites, we will assess a total of 100 hypothetical configurations of alloys.



In this example, we transmute surface Pt atoms to Au or Ir (change in nuclear charge of +/- 1) and counter-transmute Pt atoms in the bottom layers to Ir or Au (change in nuclear charge of -/+ 1). Below we construct an array with Au and Ir Atom objects to use in the `transmuter()` function.

In [12]:
charge = [1,-1]
chargelen = len(charge)
all_transmute = []
for c in charge:
    transmute_atom = Atom(metal.symbol)
    transmute_atom.number += c
    counter_transmute_atom = Atom(metal.symbol)
    counter_transmute_atom.number -= c
    all_transmute.append([counter_transmute_atom,transmute_atom])

Next, we make transmutations to Atoms objects of slab and ads from their respective CONTCARs and give labels for each new model with the `transmuted_directory_names()` function. Now we can visualize these new systems and write POSCARs to do VASP calculations for benchmarking.

In [13]:
print('Transmuted Slab Labels')
transmuted_slabs = []
dir_slab = []
for k in all_transmute:
    slab_single_image = []
    slabdir_single_image = []
    for i,c in enumerate(counter[0]):
        for j,t in enumerate(transmute[0]):
            slab_single_image.append(transmuter(slab=slab_contcar,
                                           atomdex=[c,t],
                                           trans=k))
            slab_dname = transmuted_directory_names(bdex=i,
                                           tdex=j,
                                           dexes=[c,t],
                                           atoms_array=k)
            slabdir_single_image.append(slab_dname)
            print(slab_dname)
    transmuted_slabs.append(slab_single_image)
    dir_slab.append(slabdir_single_image)

Transmuted Slab Labels
0.0.Ir0.Au12
0.1.Ir0.Au13
0.2.Ir0.Au14
0.3.Ir0.Au15
0.4.Ir0.Au8
0.5.Ir0.Au9
0.6.Ir0.Au10
0.7.Ir0.Au11
1.0.Ir1.Au12
1.1.Ir1.Au13
1.2.Ir1.Au14
1.3.Ir1.Au15
1.4.Ir1.Au8
1.5.Ir1.Au9
1.6.Ir1.Au10
1.7.Ir1.Au11
0.0.Au0.Ir12
0.1.Au0.Ir13
0.2.Au0.Ir14
0.3.Au0.Ir15
0.4.Au0.Ir8
0.5.Au0.Ir9
0.6.Au0.Ir10
0.7.Au0.Ir11
1.0.Au1.Ir12
1.1.Au1.Ir13
1.2.Au1.Ir14
1.3.Au1.Ir15
1.4.Au1.Ir8
1.5.Au1.Ir9
1.6.Au1.Ir10
1.7.Au1.Ir11


In [14]:
transmuted_ads = []
dir_ads = []
for im in range(0,len(images)):
    ads_single_image = []
    adsdir_single_image = []
    for k in all_transmute:
        ads_single = []
        adsdir_single = []
        for i,c in enumerate(counter[im]):
            for j,t in enumerate(transmute[im]):
                ads_single.append(transmuter(slab=ads_contcar[im],
                                                     atomdex=[c,t],
                                                     trans=k))
                adsdir_single.append(transmuted_directory_names(bdex=i,
                                                              tdex=j,
                                                              dexes=[c,t],
                                                              atoms_array=k))
        ads_single_image.append(ads_single)
        adsdir_single_image.append(adsdir_single)
    transmuted_ads.append(ads_single_image)
    dir_ads.append(adsdir_single_image)

import os
from ase.io import write
from comp_alchemy.vasp_setup import make_potcar,make_kpoints,make_incar,make_job_script
os.chdir('vasp_calcs/slab')
for i in range(chargelen):
    for j,s in enumerate(transmuted_slabs[i]):
        d = dir_slab[i][j]
        os.mkdir(d)
        os.chdir(d)
        home = os.getcwd()
        write('POSCAR',s,'vasp',sort=True)
        make_potcar(home,'../../../potcars')
        make_kpoints(home,d,kspacing=[8,8,1])
        make_incar(home,d,encut=350,ismear=1,sigma=0.2,ibrion=-1,nsw=0)
        make_job_script(home,d,jobnum=j,nodes=1,cores=16,cluster='smp',partition='smp',hours=6)
        os.chdir('../')
os.chdir('../../')
os.chdir('vasp_calcs/ads')
for im in images:
    os.chdir(f'{im}')
    print()
    os.chdir('../')
os.chdir('../../')

import os
from ase.io import write
from comp_alchemy.vasp_setup import make_potcar,make_kpoints,make_incar,make_job_script
os.chdir('vasp_calcs/ads')
for im in images:
    os.chdir(f'{im}')
    for i in range(chargelen):
        for j,s in enumerate(transmuted_ads[im][i]):
            d = dir_ads[im][i][j]
            os.mkdir(d)
            os.chdir(d)
            home = os.getcwd()
            write('POSCAR',s,'vasp',sort=True)
            make_kpoints(home,d,kspacing=[8,8,1])
            make_incar(home,d,encut=350,ismear=1,sigma=0.2,ibrion=-1,nsw=0)
            os.chdir('../')
    os.chdir('../')
os.chdir('../../')

### Calculating binding energy (BE) with computational alchemy
For validation purposes, we could then setup directories to run VASP calculations on our transmuted surface models. It's most convinient to analyze these calculations if they are each stored in a directory named with the labels generated above. 
Once all VASP calculations are complete, we can calculate binding energies on our transmuted surfaces using alchemy, then compare against DFT binding energies.

First we calculate the binding energy of our reference system.

In [15]:
from comp_alchemy.read_oszicar import grab_energy
adsorbate_energy = 0
slab_energy = grab_energy(oszicar='slab/trunc-OSZICAR')
ads_energy = image_energies
ref_be = [slab_energy + adsorbate_energy - ae for ae in ads_energy]

transmuted_slab_data = pd.DataFrame(columns=['deltaZ','labels','energies'])
for i in range(chargelen):
    for d in dir_slab[i]:
        transmuted_slab_data = transmuted_slab_data.append({'deltaZ':charge[i],
                                                            'labels':d,
                                                            'energies':grab_energy(oszicar=f'vasp_calcs/slab/{d}/OSZICAR')},
                                                           ignore_index=True)
print(transmuted_slab_data)

transmuted_slab_data.to_csv('transmuted_slab_energies.csv',index=False)

In [16]:
transmuted_slab_data = pd.read_csv('transmuted_slab_energies.csv',index_col='labels')
print(transmuted_slab_data)

              deltaZ   energies
labels                         
0.0.Ir0.Au12       1 -92.059727
0.1.Ir0.Au13       1 -92.066085
0.2.Ir0.Au14       1 -92.066086
0.3.Ir0.Au15       1 -92.066084
0.4.Ir0.Au8        1 -91.795667
0.5.Ir0.Au9        1 -91.726008
0.6.Ir0.Au10       1 -91.795667
0.7.Ir0.Au11       1 -91.795400
1.0.Ir1.Au12       1 -92.066084
1.1.Ir1.Au13       1 -92.059727
1.2.Ir1.Au14       1 -92.066085
1.3.Ir1.Au15       1 -92.066084
1.4.Ir1.Au8        1 -91.726247
1.5.Ir1.Au9        1 -91.795401
1.6.Ir1.Au10       1 -91.795400
1.7.Ir1.Au11       1 -91.795423
0.0.Au0.Ir12      -1 -91.938966
0.1.Au0.Ir13      -1 -91.946093
0.2.Au0.Ir14      -1 -91.946093
0.3.Au0.Ir15      -1 -91.946093
0.4.Au0.Ir8       -1 -92.406252
0.5.Au0.Ir9       -1 -92.330542
0.6.Au0.Ir10      -1 -92.406252
0.7.Au0.Ir11      -1 -92.406252
1.0.Au1.Ir12      -1 -91.946093
1.1.Au1.Ir13      -1 -91.938920
1.2.Au1.Ir14      -1 -91.946093
1.3.Au1.Ir15      -1 -91.946052
1.4.Au1.Ir8       -1 -92.330545
1.5.Au1.

In [17]:
transmuted_slab_energies = [[transmuted_slab_data['energies'][lab] for lab in dir_slab[i]] for i in range(chargelen)]

transmuted_ads_data = pd.DataFrame(columns=['image','deltaZ','labels','energies'])
for im in images:
    for i in range(chargelen):
        for d in dir_ads[im][i]:
            transmuted_ads_data = transmuted_ads_data.append({'image':im,
                                                              'deltaZ':charge[i],
                                                              'labels':d,
                                                              'energies':grab_energy(oszicar=f'vasp_calcs/ads/{im}/{d}/OSZICAR')},
                                                             ignore_index=True)
print(transmuted_ads_data)

transmuted_ads_data.to_csv('transmuted_ads_energies.csv',index=False)

In [18]:
transmuted_ads_data = pd.read_csv('transmuted_ads_energies.csv',index_col=['labels','image','deltaZ'])
print(transmuted_ads_data)

                            energies
labels       image deltaZ           
0.0.Ir0.Au12 0      1     -116.01317
0.1.Ir0.Au13 0      1     -116.03477
0.2.Ir0.Au14 0      1     -116.03094
0.3.Ir0.Au15 0      1     -116.03314
0.4.Ir0.Au8  0      1     -115.76582
0.5.Ir0.Au9  0      1     -115.69794
0.6.Ir0.Au10 0      1     -115.76566
0.7.Ir0.Au11 0      1     -115.76564
1.0.Ir1.Au12 0      1     -116.01847
1.1.Ir1.Au13 0      1     -116.02924
1.2.Ir1.Au14 0      1     -116.03068
1.3.Ir1.Au15 0      1     -116.03304
1.4.Ir1.Au8  0      1     -115.69665
1.5.Ir1.Au9  0      1     -115.76613
1.6.Ir1.Au10 0      1     -115.76613
1.7.Ir1.Au11 0      1     -115.76517
0.0.Au0.Ir12 0     -1     -116.03053
0.1.Au0.Ir13 0     -1     -116.01662
0.2.Au0.Ir14 0     -1     -116.01550
0.3.Au0.Ir15 0     -1     -116.01811
0.4.Au0.Ir8  0     -1     -116.48404
0.5.Au0.Ir9  0     -1     -116.41338
0.6.Au0.Ir10 0     -1     -116.48315
0.7.Au0.Ir11 0     -1     -116.48507
1.0.Au1.Ir12 0     -1     -116.03517
1

In [19]:
transmuted_ads_energies = [[[transmuted_ads_data['energies'][lab,im,charge[i]] for lab in dir_ads[im][i]] for i in range(chargelen)] for im in images]

In [20]:
transmuted_be = [[[transmuted_slab_energies[i][j] + adsorbate_energy - transmuted_ads_energies[im][i][j] for j in range(len(transmuted_slab_energies[i]))] for i in range(chargelen)] for im in images]
dft_del_be = [[[transmuted_be[im][i][j] - ref_be[im] for j in range(len(transmuted_be[im][i]))]for i in range(chargelen)] for im in images]

In [21]:
print(transmute)
print(counter)

[[12, 13, 14, 15, 8, 9, 10, 11], [12, 13, 14, 15, 8, 9, 10, 11], [12, 13, 14, 15, 8, 9, 10, 11], [12, 13, 14, 15, 8, 9, 10, 11], [12, 13, 14, 15, 8, 9, 10, 11], [12, 13, 14, 15, 8, 9, 10, 11], [12, 13, 14, 15, 8, 9, 10, 11], [12, 13, 14, 15, 8, 9, 10, 11], [12, 13, 14, 15, 8, 9, 10, 11], [12, 13, 14, 15, 8, 9, 10, 11]]
[[0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1]]


In [22]:
from comp_alchemy.binding_energy import alc_be
dn = [[[[alc_be(transmute=[transmute[im][k]],
               counter=[counter[im][j]],
               espdiffs=diffs[im],
               charge=charge[i])[0] for k in range(len(transmute[im]))] for j in range(len(counter[im]))] for i in range(chargelen)] for im in images]
alc_del_be = [[[[alc_be(transmute=[transmute[im][k]],
               counter=[counter[im][j]],
               espdiffs=diffs[im],
               charge=charge[i])[1] for k in range(len(transmute[im]))] for j in range(len(counter[im]))] for i in range(chargelen)] for im in images]

alc_del_be = [[[item for sublist in alc_del_be[im][i] for item in sublist] for i in range(chargelen)] for im in images]

When we use the function `alc_be()`, we are calculating the difference in BE between the hypothetical and reference systems. Below we organize the data so that we plot the actual BE for the hypothetical system.

In [34]:
BE_alc = alc_del_be
BE_dft = dft_del_be
#BE_alc = [[[alc + ref_be[im] for alc in alc_del_be[im][i]] for i in range(chargelen)] for im in images]
#BE_dft = transmuted_be

### Benchmarking computational alchemy against DFT

Now that the BEs are calculated, we can compute compute reaction barriers and evaluate the accuracy of computational alchemy against DFT. For each BE, we calculate the absolute error between alchemy and DFT below.

In [35]:
from numpy import average
ae = [[[abs(alc_del_be[im][i][j] - dft_del_be[im][i][j]) for j in range(len(alc_del_be[im][i]))] for i in range(chargelen)] for im in images]
mae = [round(average(ae[im]),3) for im in range(len(images))]

Below we also calcuate the distance between each transmute site and the binding site of CH4. This descriptor will help us analyze the data.

In [36]:
adsorbate_site = 'C'
adsorbate_indexes = [a.index for a in ads_contcar[0] if a.symbol == adsorbate_site]
adsorbate_positions = [[ads_contcar[0][d].position[0],ads_contcar[0][d].position[1],ads_contcar[0][d].position[2]]for d in adsorbate_indexes]
adsorbate_positions = [items for item in adsorbate_positions for items in item if item[2] == max([pos[2] for pos in adsorbate_positions])]
distances = [[np.sqrt((ads_contcar[0][t].position[0]-adsorbate_positions[0])**2+(ads_contcar[0][t].position[1]-adsorbate_positions[1])**2+(ads_contcar[0][t].position[2]-adsorbate_positions[2])**2) for t in transmute[0]*len(counter[0])] for i in range(chargelen)]

In [37]:
output_notebook()

im = 5

be_tooltips="""
            <p>
                <img
                src="@slab_imgs" height="100" alt="@slab_imgs" width="100"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
                ></img>
                <img
                src="@ads_imgs" height="150" alt="@ads_imgs" width="150"
                style="float: center; margin: 0px 15px 15px 0px;"
                border="2"
                ></img>
            <p>
                <span style="font-size: 25px;">Transmute Label:</span>
                <span style="font-size: 25px; font-weight: bold;">@desc</span>
            </p>
            <p>
                <span style="font-size: 25px;">DFT, Alchemy:</span>
                <span style="font-size: 25px; font-weight: bold;">@x eV, @y eV</span>
            </p>
            <p>
                <span style="font-size: 25px;">Absolute Error:</span>
                <span style="font-size: 25px; font-weight: bold;">@aerr eV</span>
            </p>
"""

pf = figure(plot_width=800, plot_height=800,tooltips=be_tooltips,title="Parity Plot for BE Predictions on NEB Images with Computational Alchemy")


pf.xaxis.axis_label = "BE(DFT)"
pf.xaxis.axis_label_text_font_style = "bold"
pf.xaxis.axis_label_text_font_size = "12pt"
pf.yaxis.axis_label = "BE(Alchemy)"
pf.yaxis.axis_label_text_font_style = "bold"
pf.yaxis.axis_label_text_font_size = "12pt"
pf.title.text_font_size = '12pt'
pf.title.align = 'center'

parity = Slope(gradient=1,y_intercept=0,)
pf.add_layout(parity)

colors = ['blue','green','red','yellow','magenta','cyan','indigo']
mx = []
mn =[]
for i in range(chargelen):
    mx.append(max(BE_dft[im][i]+BE_alc[im][i]))
    mn.append(min(BE_dft[im][i]+BE_alc[im][i]))
    source = ColumnDataSource(data=dict(
            x=[d for d in BE_dft[im][i]],
            y=[a for a in BE_alc[im][i]],
            desc=dir_slab[i],
            ads_imgs=[f'neb_images/{im}/{a}.png' for a in dir_ads[im][i]],
            slab_imgs=[f'slab/{s}.png' for s in dir_slab[i]],
            aerr=ae[im][i],
            rad=[d/50 for d in distances[i]],
            color=[colors[i]]*len(BE_alc[im][i])))
    r = pf.circle('x', 'y', radius='rad', fill_color ='color',source=source, line_color="black", line_width=2, fill_alpha=0.5, legend='delZ = '+str(charge[i]))

pf.x_range=Range1d(min(mn)*0.95,max(mx)*1.05)
pf.y_range=Range1d(min(mn)*0.95,max(mx)*1.05)

pf.legend.location = "top_left"
pf.legend.click_policy = "hide"
show(pf)

Loading BokehJS ...

In [38]:
def update(neb_image):
    neb_image = int(neb_image)
    r.data_source.data['x'] = dft_del_be[neb_image]
    r.data_source.data['y'] = alc_del_be[neb_image]
    push_notebook()

In [39]:
show(pf, notebook_handle=True)

In [40]:
interact(update, neb_image=[int(n) for n in np.linspace(0,9,10)])

C:\Users\chas0\Anaconda3\lib\site-packages\bokeh\models\sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('ads_imgs', 16), ('aerr', 16), ('color', 16), ('desc', 16), ('rad', 16), ('slab_imgs', 16), ('x', 2), ('y', 16)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
C:\Users\chas0\Anaconda3\lib\site-packages\bokeh\models\sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('ads_imgs', 16), ('aerr', 16), ('color', 16), ('desc', 16), ('rad', 16), ('slab_imgs', 16), ('x', 2), ('y', 2)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))


<function __main__.update(neb_image)>

In [41]:
dft_ea = [[BE_dft[5][i][j] - BE_dft[0][i][j] for j in range(len(BE_dft[0][i]))] for i in range(chargelen)]
alc_ea = [[BE_alc[5][i][j]-BE_alc[0][i][j] for j in range(len(BE_alc[0][i]))] for i in range(chargelen)]

In [42]:
ea_abs_error = [[abs(alc_ea[i][j] - dft_ea[i][j]) for j in range(len(alc_ea[i]))] for i in range(chargelen)]
ea_mean_abs_error = round(np.average(ea_abs_error),3)

In [43]:
for i in range(chargelen):
    for j in range(len(dft_ea[i])):
        print(dir_slab[i][j],dft_ea[i][j],alc_ea[i][j],ea_abs_error[i][j])

0.0.Ir0.Au12 -0.8077800000000224 -0.7267000000000081 0.08108000000001425
0.1.Ir0.Au13 -0.00836000000002457 -0.05310000000000059 0.04473999999997602
0.2.Ir0.Au14 -0.048569999999998004 -0.011499999999998067 0.037069999999999936
0.3.Ir0.Au15 0.007530000000002701 -0.033699999999996066 0.04122999999999877
0.4.Ir0.Au8 0.01099999999999568 0.03649999999998954 0.02549999999999386
0.5.Ir0.Au9 0.010919999999984498 0.011199999999988108 0.0002800000000036107
0.6.Ir0.Au10 -0.057390000000026475 -0.00940000000001362 0.047990000000012856
0.7.Ir0.Au11 -0.02529000000002668 -0.006100000000003547 0.019190000000023133
1.0.Ir1.Au12 -0.8228300000000104 -0.7314000000000078 0.09143000000000256
1.1.Ir1.Au13 -0.014130000000008636 -0.057800000000000296 0.04366999999999166
1.2.Ir1.Au14 -0.05345000000002642 -0.016199999999997772 0.03725000000002865
1.3.Ir1.Au15 0.00354999999998995 -0.03839999999999577 0.04194999999998572
1.4.Ir1.Au8 0.017269999999982133 0.031799999999989836 0.014530000000007703
1.5.Ir1.Au9 0.0015199

In [44]:
output_notebook()

ae_tooltips="""
            <p>
                <img
                src="@slab_imgs" height="100" alt="@slab_imgs" width="100"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
                ></img>
                <img
                src="@ads_imgs" height="150" alt="@ads_imgs" width="150"
                style="float: center; margin: 0px 15px 15px 0px;"
                border="2"
                ></img>
            <p>
                <span style="font-size: 25px;">Transmute Label:</span>
                <span style="font-size: 25px; font-weight: bold;">@desc</span>
            </p>
            <p>
                <span style="font-size: 25px;">DFT, Alchemy:</span>
                <span style="font-size: 25px; font-weight: bold;">@x eV, @y eV</span>
            </p>
            <p>
                <span style="font-size: 25px;">Absolute Error:</span>
                <span style="font-size: 25px; font-weight: bold;">@aerr eV</span>
            </p>
"""

pf = figure(plot_width=800, plot_height=800, tooltips=ae_tooltips,title="Parity Plot for Barrier Height Predictions with Computational Alchemy")


pf.xaxis.axis_label = "Ea(DFT)"
pf.xaxis.axis_label_text_font_style = "bold"
pf.xaxis.axis_label_text_font_size = "12pt"
pf.yaxis.axis_label = "Ea(Alchemy)"
pf.yaxis.axis_label_text_font_style = "bold"
pf.yaxis.axis_label_text_font_size = "12pt"
pf.title.text_font_size = '12pt'
pf.title.align = 'center'

parity = Slope(gradient=1,y_intercept=0,)
pf.add_layout(parity)

colors = ['blue','green','red','yellow','magenta','cyan','indigo']
mx = []
mn =[]
for i in range(chargelen):
    mx.append(max(dft_ea[i]+alc_ea[i]))
    mn.append(min(dft_ea[i]+alc_ea[i]))
    source = ColumnDataSource(data=dict(
            x=[d for d in dft_ea[i]],
            y=[a for a in alc_ea[i]],
            desc=dir_slab[i],
            ads_imgs=[f'neb_images/0/{a}.png' for a in dir_ads[0][i]],
            slab_imgs=[f'slab/{s}.png' for s in dir_slab[i]],
            aerr=ea_abs_error[i],
            rad=[d/50 for d in distances[i]],
            color=[colors[i]]*len(alc_ea[i])))
    r = pf.circle('x', 'y', radius='rad', fill_color ='color',source=source, line_color="black", line_width=2, fill_alpha=0.5, legend='delZ = '+str(charge[i]))

pf.x_range=Range1d(min(mn)*0.95,max(mx)*1.05)
pf.y_range=Range1d(min(mn)*0.95,max(mx)*1.05)

pf.legend.location = "top_left"
pf.legend.click_policy = "hide"
show(pf)

Loading BokehJS ...